In [ ]:
#Importaciones esenciales
import numpy as np
import pandas as pd

In [ ]:
#Cargamos lo datos históricos (datos crudos)
df = pd.read_csv('HIST_PIEZAS.csv')
df_org = df.copy(True)
df

,CG_prod,Des_prod,FECHA,CG_ORDF,FE_EMIT
0,D-12254-18,"CAPUCHON 1700 SELLADA 1""",2023-09-04 10:51:47,108969,2023-09-04 06:52:13
1,D-17226-1,"CAPUCHON 2800 PAL.D 2""X 3"" - 3"" x 4""",2023-09-04 09:56:31,106594,2023-07-18 06:58:34
2,D-15196-14,"CAMISA AR-20 1/2""-3/4""",2023-09-04 09:48:28,105004,2023-06-15 11:41:38
3,C-15245-1,"CUERPO BRIDADO AR-20 1""",2023-09-04 09:34:42,103788,2023-05-31 05:59:22
4,D-12882-14,CAJA RES.REG.RED.STD,2023-09-04 09:32:11,104694,2023-06-15 11:36:10
...,...,...,...,...,...
6015,D-11488-15,CONTRATUERCA 1800,2017-11-14 04:30:00,52322,2017-10-11 13:46:03
6016,D-14067-14,DISCO 2800 N,2017-11-14 04:20:00,52332,2017-10-11 13:46:03
6017,C-13980-1,CUERPO 2800 4L6 300x150,2017-11-14 04:00:00,52228,2017-09-22 08:02:08
6018,C-13980-1,CUERPO 2800 4L6 300x150,2017-11-13 04:00:00,52228,2017-09-22 08:02:08


In [ ]:
#Recortamos para que la fecha de emisión no incluya la hora, ya que puede ser imprecisa
df['FE_EMIT'] = df['FE_EMIT'].str[:11]
df['FECHA'] = df['FECHA'].str[:11]


In [ ]:
#Creamos una función que pasa de una fecha a un número desde el nacimiento de Cristo
def date_to_num(date:str):
    year = int(date[:4])
    month = int(date[5:7])
    day = int(date[8:])
    return year*365+month*30+day

In [ ]:
#Aplicamos las funciones para pasar las fechas relevantes a un escalar
df['FECHA'] = df['FECHA'].apply(lambda x: date_to_num(x))
df['FE_EMIT'] = df['FE_EMIT'].apply(lambda x: date_to_num(x))

### LÓGICA 1 (descontinuada)

Si FE_EMIT1<FE_ENTREGA2<FE_ENTREGA1 el producto 2 existió en simúltaneo con el producto 1  

In [ ]:
def simultaneo(fe_emit1:int, fe_entrega1:int, fe_emit2:int, fe_entrega2:int):
    return (fe_emit1<=fe_emit2 and fe_emit2<fe_entrega1) or (fe_emit1<fe_entrega2 and fe_entrega2<fe_entrega1) or (fe_emit1>=fe_emit2 and fe_entrega1<=fe_entrega2)

In [ ]:
empty = np.full(len(df), np.nan)
df.insert(3, 'SIMULTANEO', empty)

In [ ]:
df_cop = df.copy(True)

In [ ]:
df = df.drop_duplicates('PEDIDO')
df.loc[:, 'SIMULTANEO'] = df.loc[:, 'SIMULTANEO'].apply(lambda x: np.full(len(df), np.nan))
#df

In [ ]:
#Con pandas

for i in range(len(df)):
    row1 = df.iloc[i]
    for k in range(i, len(df)):
        row2 = df.iloc[k]
        simul = simultaneo(row1[5], row1[4], row2[5], row2[4])
        df.iloc[i][3][k] = simul
        df.iloc[k][3][i] = simul

In [ ]:
#Con numpy MUCHO MÁS RÁPIDO!!
data = df.to_numpy()
for i in range(len(df)):
    row1 = data[i]
    for k in range(i, len(df)):
        row2 = data[k]
        simul = simultaneo(row1[5], row1[4], row2[5], row2[4])
        data[i][3][k] = simul
        data[k][3][i] = simul

In [ ]:
df = pd.DataFrame(data=data, columns=df.columns)

In [ ]:
np.sum([i for i in df.iloc[0][3]])

611.0

In [ ]:
simultaneo(df.iloc[0][5], df.iloc[0][4], df.iloc[1][5], df.iloc[1][4])

True

In [ ]:
simultaneo(df.iloc[0][5], df.iloc[0][4], df.iloc[3][5], df.iloc[3][4])

True

In [ ]:
df[np.asarray([i for i in df.iloc[0][3]], dtype=bool)]

,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60603,NaN,NaN,1756.25
1,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60604,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60643,NaN,NaN,2262.56
3,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60644,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738759,738513,NaN,60629,NaN,NaN,2005.02
...,...,...,...,...,...,...,...,...,...,...,...
606,2838F4R2RAM1017,"2800 1.1/2""F3"" 900(RF)x300(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738519,738461,2023-04-11 13:52:21.773,60278,2023-04-11 07:10:47.777,2023-04-11 13:52:01.487,3503.0
607,1617A1R1RSM1010,"1600 1/2"" x 1"" 150(RF) x 150(RF) Tob 1/2""Std. ...",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738518,738477,NaN,60369,NaN,NaN,895.4
608,1621B1R1RSM1010,"1600 3/4"" x 1"" 150(RF) x 150(RF)",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738518,738477,NaN,60368,NaN,NaN,966.9
609,2842J1R1RSM1010,"2800 2""J3"" 150(RF)x150(RF) sellada",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",738518,738477,NaN,60370,NaN,NaN,1996.5


In [ ]:
df['DES_PROD'].str[:4].unique()
PRODUCTS = ['2600', '2800', '1800', '1700', '2100', '2400', '1600', '9300', '1100', '3000', '2805', '2405', '1804', '1900', '1500', '1400', '9200', '9100', '1200', '1805', '6800']
NAME_LEN = 7
[df['DES_PROD'].apply(lambda x: x[:4] if x[:4] in PRODUCTS else x).str[:NAME_LEN].unique(),
 len(df['DES_PROD'].apply(lambda x: x[:4] if x[:4] in PRODUCTS else x).str[:NAME_LEN].unique())]

[array(['2600', '2800', '1800', 'AC-350 ', '1700', '2100', '2400',
        'AC/R-50', '1600', 'AR-30 1', 'AR-30 2', 'AR-20 1', '9300', '1100',
        '3000', '2805', 'Junta c', '2405', '1804', '1900', '1500', '1400',
        'AR-20 2', 'VA-1000', '9200', '9100', '1200', 'AC/R-35', 'AR-50 1',
        '1805', 'AC-500 ', 'AR-10 1', 'VA-3000', 'AC-900 ', 'AR-10 3',
        '6800', 'AC/R-90', 'CALENTA', 'AR-30 4', 'AR-20 6', 'AR-10 4',
        'AR-70 1', 'AR-30 3', 'AR-70 3', 'VA-1400', 'AR-40 1', 'VA-1200',
        'PILOTO ', 'ACR-900', 'Filtro ', 'AC-200 ', 'AR-90 1'],
       dtype=object),
 52]

In [ ]:
df.drop(index=(df[df['FE_EMIT']==df['FE_ENTREGA']].index), axis=1)

In [ ]:
len(df)

501

In [ ]:
df.to_csv('HISTORICO_SIMUL.csv')

In [ ]:
df_org.drop_duplicates('PEDIDO')

,CG_PROD,DES_PROD,CANT,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.360,NaN,60603,NaN,NaN,1756.25
1,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:43.987,NaN,60604,NaN,NaN,1756.25
2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:07.730,NaN,60643,NaN,NaN,2262.56
3,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:28:08.310,NaN,60644,NaN,NaN,2262.56
4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,2023-12-04,2023-03-28 11:27:59.260,NaN,60629,NaN,NaN,2005.02
...,...,...,...,...,...,...,...,...,...,...
3505,2142H2R1RAM1013,"2100 2""H3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:06.967,2020-01-24 00:00:00.000,55960,NaN,NaN,1503.36
3506,2142J1R1RAM1012,"2100 2""J3"" 150(RF)x150(RF)",1.0,2019-10-17,2020-01-13 08:15:30.187,2020-01-24 00:00:00.000,55955,NaN,NaN,1384.32
3507,2142J2R1RAM1012,"2100 2""J3"" 300(RF)x150(RF)",1.0,2019-10-17,2020-01-10 11:04:12.380,2020-01-28 13:54:49.903,55957,NaN,NaN,1398.72
3508,2842J1R1RFM1010,"2800 2""J3"" 150(RF)x150(RF) Con Palanca",1.0,2019-03-06,2020-01-28 09:26:21.813,NaN,56407,NaN,NaN,0.00


In [ ]:
df_org.iloc[0], df_org.iloc[500]

(CG_PROD                                         2636F2R1RAM1017
 DES_PROD                         2600 1.1/2"F2" 300(RF)x150(RF)
 CANT                                                        1.0
 FE_ENTREGA                                           2023-12-04
 FE_EMIT                                 2023-03-28 11:27:43.360
 FE_CIERRE                                                   NaN
 PEDIDO                                                    60603
 FECHA_PREVISTA_FABRICACION                                  NaN
 FECHA_INICIO_REAL_FABRICACION                               NaN
 TOT_DOL                                                 1756.25
 Name: 0, dtype: object,
 CG_PROD                                     2666Q1R1RBB3117
 DES_PROD                         2600 6"Q8" 150(RF)x150(RF)
 CANT                                                    1.0
 FE_ENTREGA                                       2023-05-28
 FE_EMIT                             2023-02-28 07:17:56.483
 FE_CIERRE          

In [ ]:
df_org['DES_PROD'].str[:4].unique(), df_org['DES_PROD'].str[:7].unique()

(array(['2600', '2800', '1800', 'AC-3', '1700', '2100', '2400', 'AC/R',
        '1600', 'AR-3', 'AR-2', '9300', '1100', '3000', '2805', 'Junt',
        '2405', '1804', '1900', '1500', '1400', 'VA-1', '9200', '9100',
        '1200', 'AR-5', '1805', 'AC-5', 'AR-1', 'VA-3', 'AC-9', '6800',
        'CALE', 'AR-7', 'AR-4', 'PILO', 'ACR-', 'Filt', 'AC-2', 'AR-9'],
       dtype=object),
 array(['2600 1.', '2800 1.', '2600 3"', '1800 1"', '2800 6"', 'AC-350 ',
        '1800 2"', '2800 3"', '1700 3/', '1800 1/', '1800 3/', '2100 1"',
        '2100 1.', '1800 1.', '2800 1"', '1700 1/', '2400 3"', '2800 2"',
        'AC/R-50', '2600 1"', '1700 1"', '1800 AP', '2800 4"', '1600 3/',
        'AR-30 1', 'AR-30 2', 'AR-20 1', '9300 8"', '1100 3/', '9300 6"',
        '3000 VA', '2805 2"', '2400 1.', 'Junta c', '1800 2.', '2405 1.',
        '1600 1"', '2100 4"', '2600 4"', '1804 1"', '1804 1/', '1900 3/',
        '2600 2"', '1500 1/', '1400 1/', 'AR-20 2', 'VA-1000', '9200 3"',
        '9300 4"', '2600 

In [ ]:
df.to_csv('HISTORICO_SIMUL_1.csv')

In [ ]:
df = pd.read_csv('HISTORICO_SIMUL.csv')
df

,Unnamed: 0,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60603,NaN,NaN,1756.25
1,2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60643,NaN,NaN,2262.56
2,4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60629,NaN,NaN,2005.02
3,10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,[0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738742,738597,NaN,61272,NaN,NaN,1480.80
4,14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,[0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...,...
724,3477,2128E3R1RAM1013,"2100 1""E2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737352,737352,2020-01-24 00:00:00.000,55962,NaN,NaN,1158.72
725,3482,1821X9M9HKM1310,"1800 3/4""x1"" NPT(M)xNPT(H) Palanca Sellada",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737345,737345,2020-01-28 00:00:00.000,56269,NaN,NaN,440.00
726,3499,2836F3R1RAM3113,"2800 1.1/2""F2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-03-04 00:00:00.000,56238,NaN,NaN,2390.00
727,3500,3515C3Z1,"AC-500 1/2"" BSP H-H Punta eje libre g.indist.",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-01-20 00:00:00.000,56244,NaN,NaN,683.00


In [ ]:
del df['Unnamed: 0']

In [ ]:
df

,Unnamed: 0,CG_PROD,DES_PROD,CANT,SIMULTANEO,FE_ENTREGA,FE_EMIT,FE_CIERRE,PEDIDO,FECHA_PREVISTA_FABRICACION,FECHA_INICIO_REAL_FABRICACION,TOT_DOL
0,0,2636F2R1RAM1017,"2600 1.1/2""F2"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60603,NaN,NaN,1756.25
1,2,2638G1R1RAM1017,"2600 1.1/2""G3"" 150(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60643,NaN,NaN,2262.56
2,4,2638H2R1RAM1017,"2600 1.1/2""H3"" 300(RF)x150(RF)",1.0,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738759,738513,NaN,60629,NaN,NaN,2005.02
3,10,2838H1R1RSM1014,"2800 1.1/2""H3"" 150(RF)x150(RF) sellada",1.0,[0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738742,738597,NaN,61272,NaN,NaN,1480.80
4,14,2651K1R1RBM1016,"2600 3""K4"" 150(RF)x150(RF)",1.0,[0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,738718,738534,NaN,60765,NaN,NaN,3355.80
...,...,...,...,...,...,...,...,...,...,...,...,...
724,3477,2128E3R1RAM1013,"2100 1""E2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737352,737352,2020-01-24 00:00:00.000,55962,NaN,NaN,1158.72
725,3482,1821X9M9HKM1310,"1800 3/4""x1"" NPT(M)xNPT(H) Palanca Sellada",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737345,737345,2020-01-28 00:00:00.000,56269,NaN,NaN,440.00
726,3499,2836F3R1RAM3113,"2800 1.1/2""F2"" 600(RF)x150(RF)",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-03-04 00:00:00.000,56238,NaN,NaN,2390.00
727,3500,3515C3Z1,"AC-500 1/2"" BSP H-H Punta eje libre g.indist.",1.0,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,737318,737318,2020-01-20 00:00:00.000,56244,NaN,NaN,683.00


### Lógica 2 (vigente)

Si FE_EMIT2<FE_ENTREGA1 el producto 2 existió en simúltaneo con el producto 1, estando los productos ordenados por FE_ENTREGA

In [ ]:
#Lo primero que hacemos es limpiar el dataset eliminando los pedidos repetidos
dt = df.drop_duplicates('CG_ORDF')
dt = dt[['CG_ORDF', 'FE_EMIT', 'FECHA']].copy(True)

In [ ]:
#Luego ordenamos el dataset según las fechas de entrega que ya fueron convertidas a días
dt.sort_values('FECHA', inplace=True)
dt

,CG_ORDF,FE_EMIT,FECHA
6019,52301,736515,736545
6009,52388,736530,736549
6012,52414,736538,736549
6016,52332,736516,736549
6014,52424,736538,736549
...,...,...,...
5,104638,738590,738669
6,103770,738576,738669
7,108534,738663,738669
4,104694,738590,738669


In [ ]:
#Creamos un arreglo X donde se almacenarán los ejemplos de entrenamiento (la secuencia temporal de los productos hechos)
X = []
data = dt.to_numpy()
dt = dt.drop(dt[data[:, 1]>data[:,2]].index) #Eliminamos datos anómalos en los que la fecha de entrega se dió antes que la fecha de emisión
data = dt.to_numpy() #Convertimos los datos a numpy para mayor velocidad y simpleza
data

array([[ 52301, 736515, 736545],
       [ 52388, 736530, 736549],
       [ 52414, 736538, 736549],
       ...,
       [108534, 738663, 738669],
       [104694, 738590, 738669],
       [108969, 738669, 738669]])

In [ ]:
dt

,CG_ORDF,FE_EMIT,FECHA
6019,52301,736515,736545
6009,52388,736530,736549
6012,52414,736538,736549
6016,52332,736516,736549
6014,52424,736538,736549
...,...,...,...
5,104638,738590,738669
6,103770,738576,738669
7,108534,738663,738669
4,104694,738590,738669


In [ ]:
"""
Esta verificación lo único que hace es chequear que el último
elemento fue el último en ser entregado,
por ello esta lleno de True ya que todos los productos que lo
preceden tuvieron fechas de inicio previas a la fecha de cierre del último
"""
data[0:len(dt), 1]<=data[len(dt)-1, 2]

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
"""
Para todo el dataset lo que se hace es buscar los productos que hayan sido producidos previo la fecha de cierre del elemento i
A su vez, al avanzar de producto se verifica que la fecha de entrega de ese producto sea única, si no lo es lo que se hace es
buscar el primer producto que tenga esa fecha de entrega e incluir de ahí en adelante (ya que de otra forma productos entregados
el mismo día tendrían diferentes ventanas cuando en realidad esto no sucedio de este modo). De ser única la fecha de entrega lo que
se hace es simplemente indexar de ahí en adelante ya que los productos previos ya fueron entregados, por lo que el modelo no debería
de importarle.
"""
for i in range(len(dt)):
    index = np.argmax(data[:, 2]==data[i, 2])
    X.append(np.concatenate(([False for j in range(index)], data[index:, 1]<=data[i, 2])))

In [ ]:
#Lo que se hace es simplemente mapear las máscaras de simultaneo a los pedidos
pedidos = [dt[np.asarray(i, dtype=bool)]['CG_ORDF'].to_numpy() for i in X]

In [ ]:
#Se inserta una columna en el df que dice los productos que coexistieron
dt.insert(3, 'SIMUL', value = pedidos)

In [ ]:
dt

,CG_ORDF,FE_EMIT,FECHA,SIMUL
6019,52301,736515,736545,"[52301, 52388, 52414, 52332, 52424, 52397, 523..."
6009,52388,736530,736549,"[52388, 52414, 52332, 52424, 52397, 52322, 522..."
6012,52414,736538,736549,"[52388, 52414, 52332, 52424, 52397, 52322, 522..."
6016,52332,736516,736549,"[52388, 52414, 52332, 52424, 52397, 52322, 522..."
6014,52424,736538,736549,"[52388, 52414, 52332, 52424, 52397, 52322, 522..."
...,...,...,...,...
5,104638,738590,738669,"[106594, 105004, 103788, 107902, 104638, 10377..."
6,103770,738576,738669,"[106594, 105004, 103788, 107902, 104638, 10377..."
7,108534,738663,738669,"[106594, 105004, 103788, 107902, 104638, 10377..."
4,104694,738590,738669,"[106594, 105004, 103788, 107902, 104638, 10377..."


In [ ]:
#Esta función resume todo lo hecho hasta el momento
def logica2():
    global dt
    dt = df.drop_duplicates('CG_ORDF').copy(True)
    dt = dt[['CG_ORDF', 'FE_EMIT', 'FECHA']].copy(True)
    dt.sort_values('FECHA', inplace=True)
    X = []
    data = dt.to_numpy()
    dt = dt.drop(dt[data[:, 1]>data[:,2]].index)
    data = dt.to_numpy()
    for i in range(len(dt)):
        index = np.argmax(data[:, 2]==data[i, 2])
        X.append(np.concatenate(([False for j in range(index)], data[index:, 1]<=data[i, 2])))
    pedidos = [dt[np.asarray(i, dtype=bool)]['CG_ORDF'].to_numpy() for i in X]
    dt.insert(3, 'SIMUL', value = pedidos)
    print("Hecho!")

In [ ]:
logica2()

Hecho!


### Postprocesamiento

In [ ]:
feat_map = {
    "other":0,
    "CAPUCHON": 1,
    "CUERPO": 2,
    "CPO": 2,
    "BONETE": 3,
    "ANILLO": 4,
    "DISCO": 5,
    "CPO.TOB": 6,
    "CPO TOB": 6,
    "CONTRATUERCA": 7,
    "TOBERA": 8,
    "BUJE": 9,
    "GUIA": 10,
    "GUÍA": 10,  # Add both forms of GUIA
    "RETEN": 11,
    "VASTAGO": 12,
    "VAST": 12,
    "TORN": 13,
    "PORTADISCO": 14,
    "TUERCA": 15,
    "CABEZA": 16,
    "APOYO": 17,
    "CAJA": 18,
    "OBTURADOR": 19,
}

In [ ]:
#Inserto una columna destinada a preservar la descripción del producto que luego será de utilidad
dt.insert(4 ,'DES', np.zeros(len(dt)))

In [ ]:
#Llenamos la columna de descripción con los datos que corresponden
dt['DES'] = dt['CG_ORDF'].apply(lambda x: [df[df['CG_ORDF']==x].Des_prod.iloc[0]])
#dt

In [ ]:
#Creamos variables auxiliares a usar después
data = dt.to_numpy()
keys = feat_map.keys()

In [ ]:
#En esta parte lo que se hace es convertir todos los datos en tokens para que mi modelo luego los logre procesar sin problema

for i in range(len(data)):
    for j in range(len(data[i, 3])):
        product_index = np.argmax(data[i, 3][j] == data[:, 0]) #Buscamos el índice del producto dentro de la lista de SIMUL según el pedido
        product_name = data[product_index, 4][0] #Obtenemos la descripción/nombre del producto
        #Encodeamos la secuencia
        for k in keys:
            if k in product_name: #Si la key está dentro de la descripción mapeamos a su correspondiente encodeo
                data[i, 3][j] = feat_map[k]
                break
            else: #Si la key no está se encodea como other
                data[i, 3][j] = feat_map['other']



for i in range(len(data)):
    #Encodeamos la etiqueta Y (producto en cuestión)
    for k in keys:
        if k in data[i, 4][0]:
            data[i, 4][0] = feat_map[k]
            break
        elif k==list(keys)[-1]:
            data[i, 4][0] = feat_map['other']


In [ ]:
#Reemplazamos los datos el df por los tokenizados
dt['SIMUL'] = data[:, 3]
dt['DES'] = data[:, 4]
dt

,CG_ORDF,FE_EMIT,FECHA,X,y,SIMUL,DES
6019,52301,736515,736545,"[5, 5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, ...",[5],"[5, 5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, ...",[5]
6009,52388,736530,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5]
6012,52414,736538,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[7],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[7]
6016,52332,736516,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5]
6014,52424,736538,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[1],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[1]
...,...,...,...,...,...,...,...
5,104638,738590,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]
6,103770,738576,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]
7,108534,738663,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[0],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[0]
4,104694,738590,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]


In [ ]:
dt.rename(columns = {'SIMUL':'X'}, inplace = True)
dt.rename(columns = {'DES':'y'}, inplace = True)

In [ ]:
dt.to_csv('data_piezas.csv', index=False)

In [ ]:
pd.read_hdf('data_piezas.h5', 'df')

,CG_ORDF,FE_EMIT,FECHA,X,y,SIMUL,DES
6019,52301,736515,736545,"[5, 5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, ...",[5],"[5, 5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, ...",[5]
6009,52388,736530,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5]
6012,52414,736538,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[7],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[7]
6016,52332,736516,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[5]
6014,52424,736538,736549,"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[1],"[5, 7, 5, 1, 1, 7, 2, 0, 3, 8, 8, 3, 5, 8, 0, ...",[1]
...,...,...,...,...,...,...,...
5,104638,738590,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]
6,103770,738576,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]
7,108534,738663,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[0],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[0]
4,104694,738590,738669,"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18],"[1, 0, 2, 3, 18, 18, 0, 18, 1]",[18]
